# FUSION FINALE DES JEUX DE DONNEES

In [77]:
from utils.io import dataframe_viewer, files_search, data_merger, data_validation, data_overview, \
gen_id_from_ech, na_line_drop, find_borehole_by_position, replicate_values

import re, os
import pandas as pd
from utils.boreholes_synonyms import BOREHOLES_ENCOUNTERED
from definitions import ROOT_DIR

In [2]:
def create_df(files, check_position=True, verbose=True): # find another name for this function
    """
    create dataframes from files and test if they contain position informations
    files: list of files name
    """
    dfs = []
    i = 0
    for f in files:
        i += 1
        df = pd.read_csv(f, delimiter=',')
        if check_position:
            if 'X' in df.columns:
                df['X'] = df['X'].apply(lambda x: x.replace(',','.') if isinstance(x, str) else x)
                df['X'] = df['X'].astype('float64')
            if 'Y' in df.columns:
                df['Y'] = df['Y'].apply(lambda x: x.replace(',','.') if isinstance(x, str) else x)
                df['Y'] = df['Y'].astype('float64')
            if 'Z' in df.columns:
                df['Z'] = df['Z'].apply(lambda x: x.replace(',','.') if isinstance(x, str) else x)
                df['Z'] = df['Z'].astype('float64')
        
        df = na_line_drop(df, line_non_na=1)
        dfs.append(df)
        
        if verbose:
            if 'X' in list(df.columns): msg = ' --> Coordinates'
            else: msg = ' --> No coordinates'

            print(f"df{i} : {msg}")
    
            
    return dfs

In [3]:
csv_data_dir = ROOT_DIR + '/CF_data/Result_traitem/organisation/'
save_dir = csv_data_dir + '../fusion_finale/'

In [4]:
os.system(f"rm -rf {save_dir}") 
os.system(f"mkdir {save_dir}") 

0

In [5]:
sufx = ['sup', 'prof', 'inf', '/\dM(\*)?']
prefx = ['eau forage ']
id_reg = '\s*(?P<id>(?:^canne |Piezair )*\w*\d+\w*)\s*'

# Collecte des fichiers

In [6]:
# create my dictionary structure to retrieve good files (Keynames !!!)
files_dict={'Borehole':0,'Litho':0,'Equipm':0,'Measure':0,'Sample':0,'Unknow':0}
data_dict={'Borehole':0,'Litho':0,'Equipm':0,'Measure':0,'Sample':0,'Unknow':0}

In [7]:
files_search(csv_data_dir, files_dict, prefix='source', details=False)

Borehole  	:  15
Litho  	:  4
Equipm  	:  5
Measure  	:  13
Sample  	:  13
Unknow  	:  1


In [8]:
how=['inner', 'outer', 'left', 'right']
view = False

## Forages

In [9]:
key='Borehole'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

15 files


In [10]:
files_dict[key]

['/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Forage_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Memoris_seafile/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_1_Memoris/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Phase_2_Memoris/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Prof_contact_sol_forage/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Ext_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Pilote/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSDMA/CF_data/Result_traitem/organisation/Siterem_Result_Sol/source_merge/source_Boreholes.csv',
 '/home/yanathan/Projects/GSD

In [11]:
data_overview(files_dict[key])

Same files:[]
Files with coordinates:[1, 2, 3, 4, 8, 9, 10, 12, 14]
Files without coordinates:[0, 5, 6, 7, 11, 13]


#### $\color{green}{\textbf{Lecture et fusion}}$

In [12]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Boreholes.csv' # 1
file2= csv_data_dir + 'Phase_1_Memoris/source_merge/source_Boreholes.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 57, columns : 8, Unique values on cols: {'ID': 57}
Rows : 14, columns : 7, Unique values on cols: {'ID': 14}


(None, None)

In [13]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [14]:
dataset = mdf.copy()

In [15]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 69


#### $\color{green}{\textbf{Lecture et fusion}}$

In [16]:
file1= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Boreholes.csv' # 3
file2= csv_data_dir + 'Prof_contact_sol_forage/source_merge/source_Boreholes.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 23, columns : 8, Unique values on cols: {'ID': 23}
Rows : 8, columns : 6, Unique values on cols: {'ID': 8}


(None, None)

In [17]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [18]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [19]:
v = True
dataframe_viewer(df1, rows=3, un_val='ID', view=v)
dataframe_viewer(df2, rows=3, un_val='ID', view=v)
dataframe_viewer(dataset, rows=5, un_val='ID', view=v)
dataframe_viewer(conflict_df, rows=5, un_val='ID', view=v)

Rows : 23, columns : 8, Unique values on cols: {'ID': 23}


interactive(children=(IntSlider(value=3, description='rows', max=23, min=3, readout=False), IntSlider(value=8,…

Rows : 8, columns : 6, Unique values on cols: {'ID': 8}


interactive(children=(IntSlider(value=3, description='rows', max=8, min=3, readout=False), IntSlider(value=6, …

Rows : 78, columns : 10, Unique values on cols: {'ID': 78}


interactive(children=(IntSlider(value=5, description='rows', max=78, min=5, readout=False), IntSlider(value=10…

Rows : 0, columns : 0, Unique values on cols: {'ID': 'NA'}


interactive(children=(IntSlider(value=0, description='rows', max=0, readout=False), IntSlider(value=0, descrip…

In [20]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 78


#### $\color{green}{\textbf{Lecture et fusion}}$

In [21]:
file1= csv_data_dir + 'database_Memoris3/source_merge/source_Boreholes.csv' # 8
file2= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Boreholes.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 510, columns : 20, Unique values on cols: {'ID': 508}
Rows : 25, columns : 17, Unique values on cols: {'ID': 25}


(None, None)

In [22]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [23]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [24]:
view = True
dataframe_viewer(df1, rows=3, un_val='ID', view=view)
dataframe_viewer(df2, rows=3, un_val='ID', view=view)
dataframe_viewer(dataset, rows=5, un_val='ID', view=view)
dataframe_viewer(conflict_df, rows=5, un_val='ID', view=view)

Rows : 510, columns : 20, Unique values on cols: {'ID': 508}


interactive(children=(IntSlider(value=3, description='rows', max=510, min=3, readout=False), IntSlider(value=1…

Rows : 25, columns : 17, Unique values on cols: {'ID': 25}


interactive(children=(IntSlider(value=3, description='rows', max=25, min=3, readout=False), IntSlider(value=12…

Rows : 596, columns : 26, Unique values on cols: {'ID': 594}


interactive(children=(IntSlider(value=5, description='rows', max=596, min=5, readout=False), IntSlider(value=1…

Rows : 16, columns : 6, Unique values on cols: {'ID': 16}


interactive(children=(IntSlider(value=5, description='rows', max=16, min=5, readout=False), IntSlider(value=6,…

In [25]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Type_y':list(conflict_df.index), 'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [26]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 596


#### $\color{green}{\textbf{Lecture et fusion}}$

In [27]:
file1= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Boreholes.csv' # 11
file2= csv_data_dir + 'vUmons_logsFor/source_merge/source_Boreholes.csv' # 13

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
df2 :  --> Coordinates
Rows : 27, columns : 19, Unique values on cols: {'ID': 27}


interactive(children=(IntSlider(value=3, description='rows', max=27, min=3, readout=False), IntSlider(value=12…

Rows : 78, columns : 9, Unique values on cols: {'ID': 77}


interactive(children=(IntSlider(value=3, description='rows', max=78, min=3, readout=False), IntSlider(value=9,…

(None, None)

In [28]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [29]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [30]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Date_for_y':list(conflict_df.index), 'Type_y':list(conflict_df.index), 
                            'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [31]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 668


#### $\color{green}{\textbf{Lecture et fusion}}$

In [32]:
file1= csv_data_dir + 'Forage_Pilote/source_merge/source_Boreholes.csv' # 0
file2= csv_data_dir + 'Siterem_Ext_Pilote/source_merge/source_Boreholes.csv' # 5

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

14 NaN lines dropped
df1 :  --> No coordinates
df2 :  --> No coordinates
Rows : 58, columns : 5, Unique values on cols: {'ID': 58}


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=3, description='rows', max=58, min=3, readout=False), IntSlider(value=5,…

Rows : 10, columns : 5, Unique values on cols: {'ID': 6}


interactive(children=(IntSlider(value=3, description='rows', max=10, min=3, readout=False), IntSlider(value=5,…

(None, None)

In [33]:
df2['ID'] = df2['ID'].astype('object')

In [34]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [35]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [36]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Z_x':list(conflict_df.index), 'Type_x':list(conflict_df.index), 
                            'Long_for_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [37]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 692


#### $\color{green}{\textbf{Lecture et fusion}}$

In [38]:
file1= csv_data_dir + 'Siterem_Pilote/source_merge/source_Boreholes.csv' # 6
file2= csv_data_dir + 'Siterem_Result_Sol/source_merge/source_Boreholes.csv' # 7

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

22 NaN lines dropped
df1 :  --> No coordinates
22 NaN lines dropped
df2 :  --> No coordinates
Rows : 0, columns : 4, Unique values on cols: {'ID': 0}


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=0, description='rows', max=0, readout=False), IntSlider(value=4, descrip…

Rows : 0, columns : 4, Unique values on cols: {'ID': 0}


interactive(children=(IntSlider(value=0, description='rows', max=0, readout=False), IntSlider(value=4, descrip…

(None, None)

In [39]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [40]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [41]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 692


#### $\color{green}{\textbf{Lecture et fusion}}$

In [42]:
file1= csv_data_dir + 'observ_terrain/source_merge/source_Boreholes.csv' # 10
file2= csv_data_dir + 'result_sol_ext_pilote/source_merge/source_Boreholes.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

df1 :  --> No coordinates
19 NaN lines dropped
df2 :  --> No coordinates
Rows : 27, columns : 6, Unique values on cols: {'ID': 27}


/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


interactive(children=(IntSlider(value=3, description='rows', max=27, min=3, readout=False), IntSlider(value=6,…

Rows : 0, columns : 4, Unique values on cols: {'ID': 0}


interactive(children=(IntSlider(value=0, description='rows', max=0, readout=False), IntSlider(value=4, descrip…

(None, None)

In [43]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [44]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [45]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_pz_sol_x':[18], 'Long_pz_sol_y':[i for i in conflict_df.index if i not in [18]], 
                            'Rmq_y':list(conflict_df.index), 'Type_x':list(conflict_df.index),})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [46]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 695


#### $\color{green}{\textbf{Lecture et fusion}}$

In [47]:
file1= csv_data_dir + 'coord_ext_pilote/source_merge/source_Boreholes.csv' # 10

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

df1 :  --> Coordinates
Rows : 78, columns : 5, Unique values on cols: {'ID': 78}


interactive(children=(IntSlider(value=3, description='rows', max=78, min=3, readout=False), IntSlider(value=5,…

In [48]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['X','Y'], dist_max=1)

Conflict values present. Please resolve this manually !


In [49]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'ID_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [50]:
print(f'Dataset rows: {len(dataset)}')

Dataset rows: 757


####  $\color{blue}{\textbf{Traitement des doublons par position}}$

In [85]:
data = dataset.copy()
data['ID'] = data['ID'].apply(lambda x: str(x) if not isinstance(x, str) else x)
df_t = find_borehole_by_position(data, display=True, dist_max=0.5, bh_syn_dict=BOREHOLES_ENCOUNTERED, 
                                 save_bh_syn=False, reg_skip='\d+[a-e]\'?M?$')#, verbose=True)

Skipped 148 IDs
124 boreholes found


F1,"(153124.071, 122653.382)","['F1', '_1_', '1']"
F12,"(153021.795, 122640.338)","['F12', '_12_', '12']"
P2,"(152897.30742700002, 122614.980476)","['P2', 'P2M', 'F2M']"
P3,"(152896.001173, 122633.018193)","['P3', 'P3M', 'F3M']"
P4,"(152893.324964, 122592.103763)","['P4', 'P4M', 'F4M', '79']"
P5,"(152878.150781, 122574.804573)","['P5', 'P5M', 'F5M']"
P6,"(152858.24727300002, 122576.099336)","['P6', 'P6M', 'F6M']"
P11,"(152830.61472800002, 122633.502431)","['P11', 'P11M', 'F11M']"
P12,"(152877.815915, 122573.902564)","['P12', 'P12M', 'F12M']"
P13,"(152824.640713, 122646.922281)","['P13', 'P13M', 'F13M']"
P14,"(152815.410756, 122634.657047)","['P14', 'P14M', 'F14M']"


Updating and saving encountered boreholes dict in /home/yanathan/Projects/GSDMA/utils/boreholes_synonyms.py


In [86]:
cols = ['ID','X','Y']
dataframe_viewer(data[cols], un_val='ID'), dataframe_viewer(df_t[cols], un_val='ID')

Rows : 757, columns : 3, Unique values on cols: {'ID': 730}


interactive(children=(IntSlider(value=10, description='rows', max=757, min=10, readout=False), IntSlider(value…

Rows : 757, columns : 3, Unique values on cols: {'ID': 578}


interactive(children=(IntSlider(value=10, description='rows', max=757, min=10, readout=False), IntSlider(value…

(None, None)

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [ ]:
data_dict['Borehole'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

In [ ]:
dataset.to_csv(save_dir + 'Boreholes.csv', index=False)

###  ====================================================

# Lithologies

In [ ]:
key='Litho'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
data_overview(files_dict[key])

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'database_Memoris3/source_merge/source_Lithologies.csv' # 0
file2= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Lithologies.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [ ]:
dataset = mdf.copy()

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Lithologies.csv' # 1
file2= csv_data_dir + 'vUmons_logsFor/source_merge/source_Lithologies.csv' # 3

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'Litho_top', 'Litho_base'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [ ]:
data_dict['Litho'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

In [ ]:
dataset.to_csv(save_dir + 'Lithologies.csv', index=False)

###  ====================================================

# Echantillons

In [ ]:
key='Sample'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
data_overview(files_dict[key])

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Analysis.csv' # 2
file2= csv_data_dir + 'Liste_XY/source_merge/source_Analysis.csv' # 1

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech'], dist_max=1)

In [ ]:
dataset = mdf.copy()

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir +  'Phase_1_Memoris/source_merge/source_Analysis.csv' # 3
file2= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Analysis.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'C16-C21_y':list(conflict_df.index), 'C21-C35_y':list(conflict_df.index), 
                            'C12-C16_y':list(conflict_df.index), 'Fract_2+_y':list(conflict_df.index), 
                            'C10-C12_y':list(conflict_df.index), 'HC_tot_C10-C35_y':list(conflict_df.index), 
                            'Fract_2_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir +  'database_Memoris3/source_merge/source_Analysis.csv' # 8
file2= csv_data_dir +  'profils_sols_donnees_forages/source_merge/source_Analysis.csv' # 10

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech'], dist_max=1)

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Nappe_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir +  'Container_phyto/source_merge/source_Analysis.csv' # 0
file2= csv_data_dir +  'vUmons_logsFor/source_merge/source_Analysis.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
mdf = gen_id_from_ech(mdf, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)

In [ ]:
data = mdf
col = 'ID_ech'
for i in data.index:
    v = data.loc[i, col]
    if not pd.isnull(v) and re.search('ech', v, re.I):
        data.loc[i, 'ID'] = 'F_' + re.sub(' |.','', v,re.I)
mdf = data.copy()

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir +  'Siterem_Ext_Pilote/source_merge/source_Analysis.csv' # 5
file2= csv_data_dir +  'Siterem_Pilote/source_merge/source_Analysis.csv' # 6

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
df1['ID_ech'] = df1['ID_ech'].astype('object')

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'C16-C21_x':list(conflict_df.index), 'C12-C16_x':list(conflict_df.index), 
                            'C10-C12_x':list(conflict_df.index), 'HC_tot_C10-C35_x':list(conflict_df.index), 
                            'C21-C35_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir +  'Siterem_Result_Sol/source_merge/source_Analysis.csv' # 7
file2= csv_data_dir +  'donnees_terrain_2019/source_merge/source_Analysis.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df2['ID'] = df2['ID'].astype('object')

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID', 'ID_ech'], dist_max=1)

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir +  'result_sol_ext_pilote/source_merge/source_Analysis.csv' # 11

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID', 'ID_ech', 'Date_ech'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [ ]:
data_dict['Sample'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

In [ ]:
dataset.to_csv(save_dir + 'Analysis.csv', index=False)

###  ====================================================

## Objets inconnus

In [ ]:
key='Unknow'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir +  'database_Memoris3/source_merge/source_Unknown.csv' # 11

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

In [ ]:
dataset = df1.drop(columns='ID_date')

In [ ]:
print(f'Dataset rows: {len(dataset)}')

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [ ]:
data_dict['Unknow'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

In [ ]:
dataset.to_csv(save_dir + 'Unknown.csv', index=False)

###  ====================================================

## Equipements

In [ ]:
key='Equipm'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
data_overview(files_dict[key])

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Equipments.csv' # 0
file2= csv_data_dir + 'Phase_1_Memoris/source_merge/source_Equipments.csv' # 1

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [ ]:
dataset = mdf.copy()

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Equipments.csv' # 2
file1= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Equipments.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID'], dist_max=1)

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Equipments.csv' # 3

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

In [ ]:
df1['ID'] = df1['ID'].astype('object')

In [ ]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [ ]:
data_dict['Equipm'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID'])

In [ ]:
dataset.to_csv(save_dir + 'Equipments.csv', index=False)

###  ====================================================

# Mesures

In [ ]:
key='Measure'
dataset = pd.DataFrame()
print(len(files_dict[key]), 'files')

In [ ]:
files_dict[key]

In [ ]:
data_overview(files_dict[key])

In [ ]:
#id_reg = '\s*(?P<id>(?:^canne |Piezair )*\w*\d+\w*)\s*' # default 
id_reg = '\s*(?P<id>(?:^canne |Piezair |Drain |Moni )*\w*\d+\w*)\s*'

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'Memoris_seafile/source_merge/source_Measures.csv' #1
file2= csv_data_dir + 'Phase_1_Memoris/source_merge/source_Measures.csv' # 2

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [ ]:
dataset = mdf.copy()

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'Phase_2_Memoris/source_merge/source_Measures.csv' # 3
file2= csv_data_dir + 'database_Memoris3/source_merge/source_Measures.csv' # 7

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [ ]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

In [ ]:
for i in datafr.index:
    if re.search('FP 49 PROF', datafr.loc[i, 'ID_ech']):
        datafr.loc[i, 'ID_ech'] = 'FP49 PROF'
    elif re.search('FP 49 SUP', datafr.loc[i, 'ID_ech']):
        datafr.loc[i, 'ID_ech'] = 'FP49 SUP'

In [ ]:
datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)#, verbose=True)
mdf = datafr.copy()

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'profils_sols_donnees_forages/source_merge/source_Measures.csv' # 10
file2= csv_data_dir + 'database_Memoris3/source_merge/source_Measures.csv' # 12

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','Date_mes'], dist_max=1)

In [ ]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

In [ ]:
datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)
mdf = datafr.copy()

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [ ]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'MS_x':list(conflict_df.index), 'pH_CaCl2_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'Container_phyto/source_merge/source_Measures.csv' # 0
file2= csv_data_dir + 'Siterem_Ext_Pilote/source_merge/source_Measures.csv' # 4

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
df2['ID_ech'] = df2['ID_ech'].astype('object')

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID_ech','Date_mes'], dist_max=1)

In [ ]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

In [ ]:
#datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)
mdf = datafr.copy()

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID_ech','Date_mes'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'Siterem_Pilote/source_merge/source_Measures.csv' #5
file2= csv_data_dir + 'Siterem_Result_Sol/source_merge/source_Measures.csv' #6

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [ ]:
datafr = mdf.copy() 
found = []
for i in datafr.index:
    if pd.isnull(datafr.loc[i, 'ID_ech']):
        found.append(i)
        datafr.loc[i, 'ID_ech'] = datafr.loc[i, 'ID']
if found: print(f"{len(found)} Nan found in 'ID_ech' and fixed")

In [ ]:
datafr = gen_id_from_ech(datafr, id_ech_col='ID_ech', suffixes=sufx, prefixes=prefx, capture_regex=id_reg)
mdf = datafr.copy()

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'donnees_terrain_2019/source_merge/source_Measures.csv' # 8
file2= csv_data_dir + 'observ_terrain/source_merge/source_Measures.csv' # 9

df1, df2 = create_df([file1, file2])
dataframe_viewer(df1, rows=3, un_val='ID', view=view), dataframe_viewer(df2, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1, df2]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
mdf, conflict_df=data_merger(df1, df2, how='outer', on=['ID','Date_mes'], dist_max=1)

In [ ]:
data = mdf
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','Date_mes'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

#### $\color{green}{\textbf{Lecture et fusion}}$

In [ ]:
file1= csv_data_dir + 'result_sol_ext_pilote/source_merge/source_Measures.csv' # 11

df1 = create_df([file1])[0]
dataframe_viewer(df1, rows=3, un_val='ID', view=view)

In [ ]:
df_list = [df1]
look_for = ['Date', 'ID']
for i, df in enumerate(df_list):
    for l in look_for:
        for c in df.columns:
            if re.search(l, c, re.I): print(i, c)

In [ ]:
data = df1
dataset, conflict_df=data_merger(dataset, data, how='outer', on=['ID','ID_ech','Date_mes'], dist_max=1)

In [ ]:
print(f'Dataset rows: {len(dataset)}')

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [ ]:
data_dict['Measure'] = dataset.copy()
dataframe_viewer(dataset, rows=3, un_val=['ID','ID_ech'])

In [ ]:
dataset.to_csv(save_dir + 'Measures.csv', index=False)

###  ====================================================

# Croisement et récupération de coordonnées

In [ ]:
bh_cols = ['ID','Zone','Sous_zone','Emplacement','X','Y','Z','Zsol','Date_for','Type','Long_for','Long_pz',
           'Long_pz_sol','Diam_for','Diam_ext_pz','Diam_int_pz','Sect_crep','Ht_chbre','Ht_pz_sol','Refus',
           'Type_refus','Method','Societe','Etude', 'Resp_chantier','Rmq']

In [ ]:
data_dict['Borehole'] = data_dict['Borehole'][bh_cols]

In [ ]:
coi = ['ID','X','Y','Z','Date_for','Long_for','Diam_for']
bh_coords = data_dict['Borehole'][coi]

### Lithologies

In [ ]:
data = data_dict['Litho'].copy()
dataset, conflict_df=data_merger(data, bh_coords, how='left', on=['ID'], dist_max=1)

In [ ]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_for_x':list(conflict_df.index), 'Date_for_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
dataset.to_csv(save_dir + 'Cross_Litho-boreholes.csv', index=False)

### Analysis

In [ ]:
data = data_dict['Sample'].copy()
dataset, conflict_df=data_merger(data, bh_coords, how='left', on=['ID'], dist_max=1)

In [ ]:
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', pass_col='ID', 
                valid_dict={'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

In [ ]:
dataset.to_csv(save_dir + 'Cross_Analysis-boreholes.csv', index=False)

### measures

In [ ]:
data = data_dict['Measure'].copy()
dataset, conflict_df=data_merger(data, bh_coords, how='left', on=['ID', 'X'], dist_max=1)

In [ ]:
dataset.to_csv(save_dir + 'Cross_Measures-boreholes.csv', index=False)